In [5]:
import pandas as pd
import torch
import mysql.connector
import dgl.data
import numpy as np
from dgl import save_graphs, load_graphs
import torch as th
import gensim
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import requests
import time
#from transformers import *

#tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
#model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
input_text = "This is another example sentence."
tokenized_input = tokenizer.encode_plus(input_text, return_tensors='pt')

NameError: name 'tokenizer' is not defined

In [ ]:
outputs = model(**tokenized_input)

In [ ]:
outputs[1].shape

# Connect to database

In [7]:
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                          host='127.0.0.1',
                          database='computervision')
cursor = cnx.cursor()
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}

In [8]:
test_papers = []
with open(r'./test_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        test_papers.append(x)
in_params_test = ','.join(['%s'] * len(test_papers))

train_papers = []
with open(r'./train_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        train_papers.append(x)

total_papers = train_papers + test_papers
in_params_train = ','.join(['%s'] * len(train_papers))

val_papers = []
with open(r'./val_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        val_papers.append(x)

in_params_val = ','.join(['%s'] * len(val_papers))
in_params = ','.join(['%s'] * len(train_papers + test_papers))

## Fetch Data primary

In [9]:
def remap(x, mapping):
    return mapping.get(x, x)

In [10]:
cursor.execute("select b.ReferenceID, b.ReferencedByID from referencedBy b, Papers p where b.ReferencedByID in (%s) and b.ReferenceID = p.PaperID and p.`primary author` in (select AuthorID from affiliatedTo) and b.ReferenceID in (select PaperID from Papers) and b.ReferencedByID in (select PaperID from Papers)" % in_params, total_papers)
paper_edges = np.array(cursor.fetchall())
all_papers = np.unique(paper_edges)

remapped_papers = {all_papers[i]: i  for i in range(len(all_papers))}
vec_remap = np.vectorize(remap, otypes=[str])
paper_edges = vec_remap(paper_edges, remapped_papers)

all_papers_list = all_papers.tolist()
all_params = ','.join(['%s'] * len(all_papers_list))

cursor.execute("select p.PaperID, p.`primary author`, a.Gender, p.Title from Papers p, Authors a where p.PaperID in (%s) and p.`primary author` in (select AuthorID from affiliatedTo) and p.`primary author` = a.AuthorID" % all_params, all_papers_list)
#cursor.execute("select p.PaperID, a.AuthorID, a.Gender, p.Title from Papers p, Authors a, authoredBy ab where p.PaperID in (%s) and p.PaperID = ab.PaperID and ab.AuthoredByID = a.AuthorID" % all_params, all_papers_list) #and a.AuthorID in (select AuthorID from affiliatedTo)
author_edges = np.array(cursor.fetchall())

all_authors = np.unique(author_edges[:,1])
remapped_authors = {all_authors[i]: i  for i in range(len(all_authors))}

author_edges[:,1] = vec_remap(author_edges[:,1], remapped_authors)
author_edges[:,0] = vec_remap(author_edges[:,0], remapped_papers)

In [12]:
len(all_papers)

36167

## Create features for each node

In [15]:
model_path = './GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [36]:
def title_to_vec(title):
    title = title.replace("-", " ")
    tokens = nltk.word_tokenize(title)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    vectors = []
    for word in tokens:
        try:
            vec = model[word.lower()]
            vectors.append(vec)
        except KeyError:
            #print(word)
            continue
    title_vector = np.zeros(300)
    try:
        title_vector = sum(vectors) / len(vectors)
    except ZeroDivisionError:
        pass
    return title_vector

In [37]:
all_papers_info = pd.DataFrame(author_edges)
all_papers_info[0] = all_papers_info[0].astype(int)
all_papers_info = all_papers_info.sort_values(0)
paper_feats = []
stop_words = set(stopwords.words('english'))
for title in all_papers_info[3]:
    title_vec = title_to_vec(title)
    paper_feats.append(title_vec)
paper_feats = torch.tensor(paper_feats)

In [41]:
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}

In [42]:
def request_author_information(authors):
    resp = requests.post("https://api.semanticscholar.org/graph/v1/author/batch?fields=papers.title,affiliations,paperCount,citationCount,hIndex", headers=headers, json={"ids": authors}).json()
    return resp

def request_author_individual_dfinformation(author):
    resp = requests.get(f"https://api.semanticscholar.org/graph/v1/author/{author}?fields=papers.title,paperCount,citationCount,hIndex", headers=headers).json()
    return resp

def convert_history_to_vector(response):
    if not response:
        author_hist = np.zeros(300)
        author_hist = np.append(author_hist, [0,0,0])
        return author_hist

    author_papers = response["papers"]
    author_hist = []

    for paper in author_papers:
        paperID = paper["paperId"]
        if paperID in test_papers:
            continue
        title = paper["title"]
        title_vect = title_to_vec(title)
        if np.count_nonzero(title_vect) > 0:
            author_hist.append(title_vect)
    if len(author_hist) != 0:
        author_hist = np.mean(np.array(author_hist), axis=0)
    else:
        author_hist = np.zeros(300)

    author_paperCount = response["paperCount"]
    author_citationCount = response["citationCount"]
    author_hIndex = response["hIndex"]
    author_hist = np.append(author_hist, [author_paperCount, author_citationCount, author_hIndex])

    return author_hist

In [43]:
# Pop one element
author_bug = request_author_individual_dfinformation("47399096")
all_authors = np.delete(all_authors, 15905)

In [13]:
all_authors[15905]

'47399096'

In [44]:
index = 0
start = 100
author_feats = []
while start*index <= len(all_authors):
    print(f"{start*index}/{len(all_authors)}")
    batch_authors = all_authors[start*index: (start*index)+start]
    index += 1
    while True:
        response_batch = request_author_information(batch_authors.tolist())
        if "message" not in response_batch:
            break
        time.sleep(3)

    for response in response_batch:
        author_hist = convert_history_to_vector(response)
        author_feats.append(author_hist)
        if author_hist.shape[0] != 303:
            print(response)
            print(author_hist.shape)

0/50766
100/50766
200/50766
300/50766
400/50766
500/50766
600/50766
700/50766
800/50766
900/50766
1000/50766
1100/50766
1200/50766
1300/50766
1400/50766
1500/50766
1600/50766
1700/50766
1800/50766
1900/50766
2000/50766
2100/50766
2200/50766
2300/50766
2400/50766
2500/50766
2600/50766
2700/50766
2800/50766
2900/50766
3000/50766
3100/50766
3200/50766
3300/50766
3400/50766
3500/50766
3600/50766
3700/50766
3800/50766
3900/50766
4000/50766
4100/50766
4200/50766
4300/50766
4400/50766
4500/50766
4600/50766
4700/50766
4800/50766
4900/50766
5000/50766
5100/50766
5200/50766
5300/50766
5400/50766
5500/50766
5600/50766
5700/50766
5800/50766
5900/50766
6000/50766
6100/50766
6200/50766
6300/50766
6400/50766
6500/50766
6600/50766
6700/50766
6800/50766
6900/50766
7000/50766
7100/50766
7200/50766
7300/50766
7400/50766
7500/50766
7600/50766
7700/50766
7800/50766
7900/50766
8000/50766
8100/50766
8200/50766
8300/50766
8400/50766
8500/50766
8600/50766
8700/50766
8800/50766
8900/50766
9000/50766
9100/50766



KeyboardInterrupt



In [13]:
author_bug_vector = convert_history_to_vector(author_bug)
author_feats = np.insert(author_feats, 15905, author_bug_vector, axis=0)

In [19]:
author_feats_vec = torch.tensor(author_feats)

In [36]:
author_feats_vec[:,[300,301,302]].shape

torch.Size([18355, 3])

In [41]:
author_feats_vec[:,[300,301,302]] = torch.nn.functional.normalize(author_feats_vec[:,[300,301,302]], dim=0)

In [43]:
cursor.execute("select distinct p.`primary author` from Papers p where p.PaperID in (%s)" % in_params, total_papers)
authors_in_question = np.array(cursor.fetchall()).flatten()


## Fetch the Gender

In [44]:
all_params_gender = ','.join(['%s'] * len(all_authors))

In [45]:
cursor.execute("select distinct AuthorID, Gender from Authors a where a.AuthorID in (%s)" % all_params_gender, all_authors.tolist())
gender_edges = np.array(cursor.fetchall())

In [46]:
gender_edges[:,0] = vec_remap(gender_edges[:,0], remapped_authors)

## Fetch the affiliation

In [47]:
cursor.execute("select p.PaperID, p.`primary author`, b.AffiliationID, b.Country from Papers p, affiliatedTo af, Affiliations b where p.PaperID in (%s) and p.`primary author` in (select AuthorID from affiliatedTo) and p.`primary author` = af.AuthorID and af.affiliatedTo = b.AffiliationID" % all_params, all_papers_list)
author_info = np.array(cursor.fetchall())

all_affiliations = np.unique(author_info[:,2])
remapped_affiliations = {all_affiliations[i]: i  for i in range(len(all_affiliations))}

all_countries = np.unique(author_info[:,3])
remapped_countries = {all_countries[i]: i  for i in range(len(all_countries))}

author_info[:,2] = vec_remap(author_info[:,2], remapped_affiliations)
author_info[:,3] = vec_remap(author_info[:,3], remapped_countries)
author_info[:,1] = vec_remap(author_info[:,1], remapped_authors)
author_info[:,0] = vec_remap(author_info[:,0], remapped_papers)

affiliation_edges = author_info[:,[2,1]]
affiliation_edges = np.unique(affiliation_edges, axis=0)

country_edges = author_info[:,[3,2]]
country_edges = np.unique(country_edges, axis=0)

# Fetch data

In [18]:
cursor.execute("select * from referencedBy b where b.ReferenceID in (select PaperID from Papers) and b.ReferencedByID in (select PaperID from Papers)")
paper_edges = np.array(cursor.fetchall())
all_papers = np.unique(paper_edges)

remapped_papers = {all_papers[i]: i  for i in range(len(all_papers))}
vec_remap = np.vectorize(remap, otypes=[str])
paper_edges = vec_remap(paper_edges, remapped_papers)

cursor.execute("select a.PaperID, a.AuthoredByID, p.PaperID, p.PaperID from authoredBy a, Papers p where a.PaperID in (select PaperID from Papers) and (a.PaperID in (select ReferenceID from referencedBy) or a.PaperID in (select ReferencedByID from referencedBy)) and a.PaperID in (select p.PaperID from Papers p where p.Leaf = FALSE) and a.AuthoredByID in (select a.AuthorID from affiliatedTo a) and a.PaperID = p.PaperID")

author_edges = np.array(cursor.fetchall())

all_authors = np.unique(author_edges[:,1])
remapped_authors = {all_authors[i]: i  for i in range(len(all_authors))}

author_edges[:,1] = vec_remap(author_edges[:,1], remapped_authors)
author_edges[:,0] = vec_remap(author_edges[:,0], remapped_papers)

In [19]:
cursor.execute("select distinct b.AuthoredByID from Papers p, authoredBy b where p.Leaf = False and p.PaperID in (%s) and p.PaperID = b.PaperID and b.AuthoredByID in (select a.AuthorID from affiliatedTo a)" % in_params, train_papers+test_papers)
authors_in_question = np.array(cursor.fetchall()).flatten()

# Paper data

Since we use only paper information, we remove all other "authoredBy" edges

In [49]:
remapped_test_papers = th.tensor([remapped_papers[idx] for idx in test_papers])
remapped_val_papers = th.tensor([remapped_papers[idx] for idx in val_papers])
remapped_train_papers = th.tensor([remapped_papers[idx] for idx in train_papers])
remapped_authors_in_question = th.tensor([remapped_authors[idx] for idx in authors_in_question])

In [50]:
hetero_graph = dgl.heterograph(
    {
        ('paper', 'authored', 'author'): (author_edges[:,0].astype(int), author_edges[:,1].astype(int)),
        ('author', "writes", "paper"): (author_edges[:,1].astype(int), author_edges[:,0].astype(int)),
        ('paper', 'cites', 'paper'): (paper_edges[:,0].astype(int), paper_edges[:,1].astype(int)),
        ('gender', 'gendered', 'author'): (gender_edges[:,1].astype(int), gender_edges[:,0].astype(int)),
        ('paper', 'authored_gender', 'gender'): (author_edges[:,0].astype(int), author_edges[:,2].astype(int)),
        ('paper', 'authored_affiliation', 'affiliation'): (author_info[:,0].astype(int), author_info[:,2].astype(int)),
        ('affiliation', 'affiliated', 'author'): (affiliation_edges[:,0].astype(int), affiliation_edges[:,1].astype(int)),
        ('paper', 'authored_country', 'country'): (author_info[:,0].astype(int), author_info[:,3].astype(int)),
        ('country', 'contains', 'affiliation'): (country_edges[:,0].astype(int), country_edges[:,1].astype(int))
    }
)
hetero_graph.nodes['author'].data['feature'] = author_feats_vec
hetero_graph.nodes['paper'].data['feature'] = paper_feats

In [51]:
"""
edges = hetero_graph.edges(etype="authored")
author_mask = ~torch.isin(edges[1], remapped_authors_in_question)
paper_edges = edges[0][author_mask]
author_edges = edges[1][author_mask]
author_ids_remove = hetero_graph.edge_ids(paper_edges, author_edges, etype="authored")
hetero_graph = dgl.remove_edges(hetero_graph, author_ids_remove, etype="authored")
"""

'\nedges = hetero_graph.edges(etype="authored")\nauthor_mask = ~torch.isin(edges[1], remapped_authors_in_question)\npaper_edges = edges[0][author_mask]\nauthor_edges = edges[1][author_mask]\nauthor_ids_remove = hetero_graph.edge_ids(paper_edges, author_edges, etype="authored")\nhetero_graph = dgl.remove_edges(hetero_graph, author_ids_remove, etype="authored")\n'

### Split into Training, Validation, Test set

#### Train set

In [52]:
edges = hetero_graph.edges(etype="authored")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = hetero_graph.edge_ids(train_src, train_dst, etype="authored")
train_hetero_graph = dgl.remove_edges(hetero_graph, train_ids, etype="authored")

train_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 2385, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 36167, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [53]:
edges = train_hetero_graph.edges(etype="authored_gender")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = train_hetero_graph.edge_ids(train_src, train_dst, etype="authored_gender")
train_hetero_graph = dgl.remove_edges(train_hetero_graph, train_ids, etype="authored_gender")

train_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 2385, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 2385, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [54]:
edges = train_hetero_graph.edges(etype="authored_affiliation")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = train_hetero_graph.edge_ids(train_src, train_dst, etype="authored_affiliation")
train_hetero_graph = dgl.remove_edges(train_hetero_graph, train_ids, etype="authored_affiliation")

train_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 2385, ('paper', 'authored_affiliation', 'affiliation'): 2385, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 2385, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [55]:
edges = train_hetero_graph.edges(etype="authored_country")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = train_hetero_graph.edge_ids(train_src, train_dst, etype="authored_country")
train_hetero_graph = dgl.remove_edges(train_hetero_graph, train_ids, etype="authored_country")

train_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 2385, ('paper', 'authored_affiliation', 'affiliation'): 2385, ('paper', 'authored_country', 'country'): 2468, ('paper', 'authored_gender', 'gender'): 2385, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

# Validation Set

In [56]:
edges = hetero_graph.edges(etype="authored")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = hetero_graph.edge_ids(val_src, val_dst, etype="authored")
val_hetero_graph = dgl.remove_edges(hetero_graph, val_ids, etype="authored")
val_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 36167, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [57]:
edges = val_hetero_graph.edges(etype="authored_gender")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = val_hetero_graph.edge_ids(val_src, val_dst, etype="authored_gender")
val_hetero_graph = dgl.remove_edges(val_hetero_graph, val_ids, etype="authored_gender")
val_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 300, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [58]:
edges = val_hetero_graph.edges(etype="authored_affiliation")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = val_hetero_graph.edge_ids(val_src, val_dst, etype="authored_affiliation")
val_hetero_graph = dgl.remove_edges(val_hetero_graph, val_ids, etype="authored_affiliation")
val_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 300, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 300, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [59]:
edges = val_hetero_graph.edges(etype="authored_country")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = val_hetero_graph.edge_ids(val_src, val_dst, etype="authored_country")
val_hetero_graph = dgl.remove_edges(val_hetero_graph, val_ids, etype="authored_country")
val_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 300, ('paper', 'authored_country', 'country'): 383, ('paper', 'authored_gender', 'gender'): 300, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

## Test set

In [60]:
edges = hetero_graph.edges(etype="authored")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = hetero_graph.edge_ids(val_src, val_dst, etype="authored")
test_hetero_graph = dgl.remove_edges(hetero_graph, val_ids, etype="authored")
test_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 1006, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 36167, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [61]:
edges = test_hetero_graph.edges(etype="authored_gender")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = test_hetero_graph.edge_ids(val_src, val_dst, etype="authored_gender")
test_hetero_graph = dgl.remove_edges(test_hetero_graph, val_ids, etype="authored_gender")
test_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 1006, ('paper', 'authored_affiliation', 'affiliation'): 36353, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 1006, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [62]:
edges = test_hetero_graph.edges(etype="authored_affiliation")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = test_hetero_graph.edge_ids(val_src, val_dst, etype="authored_affiliation")
test_hetero_graph = dgl.remove_edges(test_hetero_graph, val_ids, etype="authored_affiliation")
test_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 1006, ('paper', 'authored_affiliation', 'affiliation'): 1006, ('paper', 'authored_country', 'country'): 36353, ('paper', 'authored_gender', 'gender'): 1006, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [63]:
edges = test_hetero_graph.edges(etype="authored_country")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = test_hetero_graph.edge_ids(val_src, val_dst, etype="authored_country")
test_hetero_graph = dgl.remove_edges(test_hetero_graph, val_ids, etype="authored_country")
test_hetero_graph

Graph(num_nodes={'affiliation': 2345, 'author': 18355, 'country': 88, 'gender': 2, 'paper': 36167},
      num_edges={('affiliation', 'affiliated', 'author'): 18459, ('author', 'writes', 'paper'): 36167, ('country', 'contains', 'affiliation'): 2345, ('gender', 'gendered', 'author'): 18354, ('paper', 'authored', 'author'): 1006, ('paper', 'authored_affiliation', 'affiliation'): 1006, ('paper', 'authored_country', 'country'): 1089, ('paper', 'authored_gender', 'gender'): 1006, ('paper', 'cites', 'paper'): 139165},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [69]:
save_graphs("./graphs/hetero_graphs_primary_word2vec_all.bin", [train_hetero_graph, val_hetero_graph, test_hetero_graph])